In [ ]:
!pip install numpy pandas tensorflow torch transformers datasets scikit-learn matplotlib seaborn nltk



In [ ]:
import pandas as pd

df = pd.read_csv("/content/sample_data/annotated_bicodemix_publicsa_v2.csv")

df.head()


,id,post/keyword,comment/tweet,username,like count,replied to,reply count,2nd level comment/ reply,time created,majority_sent,majority_sarc,lang_id
0,28987,Taiping Perak Banjir 28/08/2022 #azrulazim352 ...,sius la awk..bnjir sana..kwn xckp ape pon??,33d2a31d-c13,1,---,1,No,29/8/2022,neutral,notsarc,my
1,28988,Taiping Perak Banjir 28/08/2022 #azrulazim352 ...,"srikota, taman sg mas, air kuning semua banjir...",61957789-224,0,33d2a31d-c13,---,Yes,29/8/2022,neutral,notsarc,my
2,28989,Taiping Perak Banjir 28/08/2022 #azrulazim352 ...,massyaallah.. . dasyatnya bnjir kali nie kawan...,cffa43d1-936,2,---,1,No,29/8/2022,negative,notsarc,my
3,28990,banjir simpang 4 benoni #banjir2022 Publish: 1...,Simpang p papar kan?,c3128b6f-2e5,2,---,0,No,2d ago,neutral,notsarc,my
4,28991,banjir simpang 4 benoni #banjir2022 Publish: 1...,nasib baik lah sempat limpas,813bbc92-ca2,1,---,0,No,2d ago,positive,notsarc,my


In [ ]:
print(df.info())
print(df.describe())
print(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        10000 non-null  int64 
 1   post/keyword              10000 non-null  object
 2   comment/tweet             10000 non-null  object
 3   username                  10000 non-null  object
 4   like count                10000 non-null  object
 5   replied to                7969 non-null   object
 6   reply count               10000 non-null  object
 7   2nd level comment/ reply  7950 non-null   object
 8   time created              10000 non-null  object
 9   majority_sent             9838 non-null   object
 10  majority_sarc             10000 non-null  object
 11  lang_id                   10000 non-null  object
dtypes: int64(1), object(11)
memory usage: 937.6+ KB
None
                 id
count  10000.000000
mean   34596.150000
std     3228.88350

In [ ]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

english_stopwords = set(stopwords.words('english'))
malay_stopwords = {"yang", "untuk", "ini", "dengan", "dari", "dan", "juga", "oleh", "ke", "di", "pada", "ada"}
stop_words = english_stopwords.union(malay_stopwords)


def clean_text(text):
    if not isinstance(text, str) or text.strip() == "":
        return "empty_text"

    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'@\w+|\#\w+', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text if text.strip() != "" else "empty_text"

if "comment/tweet" in df.columns:
    df["clean_text"] = df["comment/tweet"].astype(str).apply(clean_text)
    print(df[["comment/tweet", "clean_text"]].head(10))
else:
    print("Error: 'comment/tweet' column not found in dataset!")



                                       comment/tweet  \
0        sius la awk..bnjir sana..kwn xckp ape pon??   
1  srikota, taman sg mas, air kuning semua banjir...   
2  massyaallah.. . dasyatnya bnjir kali nie kawan...   
3                               Simpang p papar kan?   
4                       nasib baik lah sempat limpas   
5                                     tadi jam 4 ptg   
6  baru jga sya limpas sini tdi ptang jam 3 seblu...   
7  nasib dapat limpas tadi.di sabindo pun banjir ...   
8                                   Na mcm mn mulik?   
9  semoga kita semua dilindugi oleh allah aamiinn...   

                                          clean_text  
0              sius la awkbnjir sanakwn xckp ape pon  
1  srikota taman sg mas air kuning semua banjir y...  
2  massyaallah dasyatnya bnjir kali nie kawan hurmmm  
3                                simpang p papar kan  
4                       nasib baik lah sempat limpas  
5                                     tadi jam 4 ptg 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.preprocessing import LabelEncoder

sentiment_encoder = LabelEncoder()
df["sentiment_label"] = sentiment_encoder.fit_transform(df["majority_sent"])

sarcasm_encoder = LabelEncoder()
df["sarcasm_label"] = sarcasm_encoder.fit_transform(df["majority_sarc"])

print("Sentiment Label Mapping:", dict(zip(sentiment_encoder.classes_, sentiment_encoder.transform(sentiment_encoder.classes_))))
print("Sarcasm Label Mapping:", dict(zip(sarcasm_encoder.classes_, sarcasm_encoder.transform(sarcasm_encoder.classes_))))

print(df[["majority_sent", "sentiment_label", "majority_sarc", "sarcasm_label"]].head(10))


Sentiment Label Mapping: {'negative': 0, 'neutral': 1, 'positive': 2, nan: 3}
Sarcasm Label Mapping: {'notsarc': 0, 'sarc': 1}
  majority_sent  sentiment_label majority_sarc  sarcasm_label
0       neutral                1       notsarc              0
1       neutral                1       notsarc              0
2      negative                0       notsarc              0
3       neutral                1       notsarc              0
4      positive                2       notsarc              0
5       neutral                1       notsarc              0
6       neutral                1       notsarc              0
7       neutral                1       notsarc              0
8       neutral                1       notsarc              0
9      positive                2       notsarc              0


In [ ]:
print("Unique Sentiment Labels:", sorted(df["sentiment_label"].unique()))
print("Unique Sarcasm Labels:", sorted(df["sarcasm_label"].unique()))

Unique Sentiment Labels: [0, 1, 2, 3]
Unique Sarcasm Labels: [0, 1]


In [ ]:
from sklearn.preprocessing import LabelEncoder

sentiment_encoder = LabelEncoder()
df["sentiment_label"] = sentiment_encoder.fit_transform(df["majority_sent"])

print("Sentiment Mapping:", dict(zip(sentiment_encoder.classes_, sentiment_encoder.transform(sentiment_encoder.classes_))))


Sentiment Mapping: {'negative': 0, 'neutral': 1, 'positive': 2, nan: 3}


In [ ]:
df = df[df["sentiment_label"].isin([0, 1, 2])]


In [ ]:
print("Unique Sentiment Labels After Fix:", sorted(df["sentiment_label"].unique()))


Unique Sentiment Labels After Fix: [0, 1, 2]


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X = df["clean_text"]
y_sentiment = np.array(df["sentiment_label"]).reshape(-1)
y_sarcasm = np.array(df["sarcasm_label"]).reshape(-1)


X_train, X_test, y_train_sentiment, y_test_sentiment = train_test_split(X, y_sentiment, test_size=0.2, random_state=42)
X_train, X_test, y_train_sarcasm, y_test_sarcasm = train_test_split(X, y_sarcasm, test_size=0.2, random_state=42)


print("Train Sentiment Shape:", y_train_sentiment.shape)
print("Test Sentiment Shape:", y_test_sentiment.shape)
print("Train Sarcasm Shape:", y_train_sarcasm.shape)
print("Test Sarcasm Shape:", y_test_sarcasm.shape)



Train Sentiment Shape: (7870,)
Test Sentiment Shape: (1968,)
Train Sarcasm Shape: (7870,)
Test Sarcasm Shape: (1968,)


In [ ]:

print("Training set size:", len(X_train))
print("Testing set size:", len(X_test))

print("\nSample training data:")
print(X_train[:5])

print("\nSample labels (Sentiment):")
print(y_train_sentiment[:5])

print("\nSample labels (Sarcasm):")
print(y_train_sarcasm[:5])


Training set size: 7870
Testing set size: 1968

Sample training data:
7414    e5e alhamdulillah bangini yg dirisaukan kalau ...
8536    begini sekali tekanan memikirkan krisis ekonom...
1260                                       zaman among us
1201    langkawi pon masih tutup kaw perlancongan rasa...
7248                                           good night
Name: clean_text, dtype: object

Sample labels (Sentiment):
[0 0 1 0 1]

Sample labels (Sarcasm):
[0 1 0 0 0]


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

def tokenize_data(texts):
    return tokenizer(texts.tolist(), padding=True, truncation=True, max_length=128, return_tensors="tf")

train_encodings = tokenize_data(X_train)
test_encodings = tokenize_data(X_test)


In [ ]:
print("Train Input IDs Shape:", train_encodings["input_ids"].shape)
print("Train Attention Mask Shape:", train_encodings["attention_mask"].shape)
print("Test Input IDs Shape:", test_encodings["input_ids"].shape)
print("Test Attention Mask Shape:", test_encodings["attention_mask"].shape)


Train Input IDs Shape: (7870, 102)
Train Attention Mask Shape: (7870, 102)
Test Input IDs Shape: (1968, 84)
Test Attention Mask Shape: (1968, 84)


In [ ]:
#best model
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import tensorflow as tf
from transformers import TFBertModel
from sklearn.utils.class_weight import compute_class_weight

# Download necessary NLTK data
nltk.download('stopwords')

# Define stopwords
english_stopwords = set(stopwords.words('english'))
malay_stopwords = {"yang", "untuk", "ini", "dengan", "dari", "dan", "juga", "oleh", "ke", "di", "pada", "ada"}
stop_words = english_stopwords.union(malay_stopwords)

# Function to clean the text
def clean_text(text):
    if not isinstance(text, str) or text.strip() == "":
        return "empty_text"
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'@\w+|\#\w+', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text if text.strip() != "" else "empty_text"

# Load the data
df = pd.read_csv("/content/annotated_bicodemix_publicsa_v2.csv")

# Preprocess the data
df["clean_text"] = df["comment/tweet"].astype(str).apply(clean_text)

# Encode sentiment and sarcasm labels
sentiment_encoder = LabelEncoder()
df["sentiment_label"] = sentiment_encoder.fit_transform(df["majority_sent"])
df = df[df["sentiment_label"].isin([0, 1, 2])]

# Split the data
X = df["clean_text"]
y_sentiment = np.array(df["sentiment_label"]).reshape(-1)
X_train, X_test, y_train_sentiment, y_test_sentiment = train_test_split(X, y_sentiment, test_size=0.2, random_state=42)

# Tokenize the text
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
def tokenize_data(texts):
    return tokenizer(texts.tolist(), padding=True, truncation=True, max_length=128, return_tensors="tf")
train_encodings = tokenize_data(X_train)
test_encodings = tokenize_data(X_test)

# Define the model
class SentimentModel(tf.keras.Model):
    def __init__(self, model_name="bert-base-multilingual-cased"):
        super(SentimentModel, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.sentiment_output = tf.keras.layers.Dense(3, activation="softmax", name="sentiment_output")

    def call(self, inputs):
        bert_output = self.bert(inputs)[1]
        sentiment = self.sentiment_output(bert_output)
        return {"sentiment_output": sentiment}

model = SentimentModel()

# Compile and train the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss={"sentiment_output": "sparse_categorical_crossentropy"},
    metrics=["accuracy"]
)

history = model.fit(
    x={"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"]},
    y={"sentiment_output": y_train_sentiment.astype("int64")},
    validation_data=(
        {"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"]},
        {"sentiment_output": y_test_sentiment.astype("int64")}
    ),
    epochs=3,  # Reduced epochs for demonstration
    batch_size=8
)

# Function to predict sentiment of a given text
def predict_text(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors="tf")
    predictions = model.predict({"input_ids": inputs["input_ids"], "attention_mask": inputs["attention_mask"]})
    sentiment_pred = predictions["sentiment_output"]
    sentiment_label = sentiment_pred.argmax(axis=-1)[0]
    return sentiment_label

# Example usage
sample_text = "Wow, this is the best government ever!"
sentiment = predict_text(sample_text)
print(f"Predicted Sentiment: {sentiment}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

Epoch 1/3
984/984 [==============================] - 245s 204ms/step - loss: 0.9703 - accuracy: 0.5221 - val_loss: 0.9300 - val_accuracy: 0.5376
Epoch 2/3
984/984 [==============================] - 205s 208ms/step - loss: 0.7858 - accuracy: 0.6576 - val_loss: 0.8563 - val_accuracy: 0.6138
Epoch 3/3
1/1 [==============================] - 3s 3s/step
Predicted Sentiment: 2


In [ ]:
# prompt: write a code for saving this model

import tensorflow as tf

# Save the entire model
model.save('my_sentiment_model')

# You can then load it later using:
# reloaded_model = tf.keras.models.load_model('my_sentiment_model')

# Alternatively, save only the weights:
# model.save_weights('my_sentiment_model_weights')

# And load them later:
# model.load_weights('my_sentiment_model_weights') # Make sure to initialize the model first

# For Colab, download the saved model
from google.colab import files
!zip -r my_sentiment_model.zip my_sentiment_model
files.download('my_sentiment_model.zip')


  adding: my_sentiment_model/ (stored 0%)
  adding: my_sentiment_model/saved_model.pb (deflated 92%)
  adding: my_sentiment_model/keras_metadata.pb (deflated 95%)
  adding: my_sentiment_model/assets/ (stored 0%)
  adding: my_sentiment_model/variables/ (stored 0%)
  adding: my_sentiment_model/variables/variables.index (deflated 79%)
  adding: my_sentiment_model/variables/variables.data-00000-of-00001 (deflated 37%)
  adding: my_sentiment_model/fingerprint.pb (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>